# Sup Synoptic

https://developers.synopticdata.com/mesonet/v2/stations/timeseries/

## CONVENTION 

All saved times (have to fix this) are ZULU time. But for processing, they are converted to location time.

* [ ] Don't forget to add the table init!!!11

In [1]:
import requests
import configparser
from datetime import datetime, timedelta
import pytz
import sqlite3

import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read("../config")

['../config']

In [3]:
token = config["synoptic"]["token"]

In [4]:
base_url = "https://api.synopticdata.com/v2/stations/timeseries?"

## start and end

Let's just pretend that we want to get yesterday's high temperature

YYYYmmddHHMM. Where YYYY is year, mm is month, dd is day, HH is hour, and MM is minutes

start=201306011800&end=201306021215

In [5]:
yesterday = datetime.now() - timedelta(days=1)
yesterday

datetime.datetime(2022, 11, 9, 16, 17, 52, 160912)

In [6]:
yesterday = yesterday.strftime("%Y%m%d")

In [7]:
yesterday_start = yesterday + "0001"
yesterday_end = yesterday + "2359"

## station ID

stid=knyc

In [8]:
stid='knyc'

In [9]:
final_url = f"{base_url}&stid={stid}&start={yesterday_start}&units=temp|F&end={yesterday_end}&showemptystations=1&token={token}"
final_url

'https://api.synopticdata.com/v2/stations/timeseries?&stid=knyc&start=202211090001&units=temp|F&end=202211092359&showemptystations=1&token=933a6b2f4c4e46898f8e80c91225fd04'

In [10]:
r = requests.get(final_url)

In [11]:
weather = r.json()
weather.keys()

dict_keys(['UNITS', 'QC_SUMMARY', 'STATION', 'SUMMARY'])

In [12]:
weather["STATION"][0].keys()

dict_keys(['STATUS', 'MNET_ID', 'PERIOD_OF_RECORD', 'ELEVATION', 'NAME', 'STID', 'SENSOR_VARIABLES', 'ELEV_DEM', 'LONGITUDE', 'UNITS', 'STATE', 'OBSERVATIONS', 'RESTRICTED', 'QC_FLAGGED', 'LATITUDE', 'TIMEZONE', 'ID'])

In [13]:
weather["STATION"][0]["OBSERVATIONS"].keys()

dict_keys(['altimeter_set_1', 'wind_cardinal_direction_set_1d', 'wind_gust_set_1', 'wind_chill_set_1d', 'weather_summary_set_1d', 'sea_level_pressure_set_1', 'sea_level_pressure_set_1d', 'cloud_layer_1_code_set_1', 'cloud_layer_1_set_1d', 'air_temp_low_6_hour_set_1', 'air_temp_high_24_hour_set_1', 'pressure_set_1d', 'pressure_tendency_set_1', 'metar_set_1', 'relative_humidity_set_1', 'wind_speed_set_1', 'air_temp_set_1', 'air_temp_high_6_hour_set_1', 'date_time', 'dew_point_temperature_set_1d', 'visibility_set_1', 'dew_point_temperature_set_1', 'air_temp_low_24_hour_set_1', 'metar_origin_set_1', 'wind_direction_set_1'])

In [14]:
date_time = weather["STATION"][0]["OBSERVATIONS"]['date_time']
date_time

['2022-11-09T00:51:00Z',
 '2022-11-09T01:51:00Z',
 '2022-11-09T02:51:00Z',
 '2022-11-09T03:51:00Z',
 '2022-11-09T04:51:00Z',
 '2022-11-09T05:51:00Z',
 '2022-11-09T06:51:00Z',
 '2022-11-09T07:51:00Z',
 '2022-11-09T08:51:00Z',
 '2022-11-09T09:51:00Z',
 '2022-11-09T10:51:00Z',
 '2022-11-09T11:51:00Z',
 '2022-11-09T12:51:00Z',
 '2022-11-09T13:51:00Z',
 '2022-11-09T14:51:00Z',
 '2022-11-09T15:51:00Z',
 '2022-11-09T16:51:00Z',
 '2022-11-09T17:51:00Z',
 '2022-11-09T18:51:00Z',
 '2022-11-09T19:51:00Z',
 '2022-11-09T20:51:00Z',
 '2022-11-09T21:51:00Z',
 '2022-11-09T22:51:00Z',
 '2022-11-09T23:51:00Z']

In [15]:
utc = pytz.timezone("UTC")

In [16]:
nyc = pytz.timezone("America/New_York")

In [17]:
str(utc.localize(datetime.strptime(date_time[0], "%Y-%m-%dT%H:%M:%SZ")).astimezone(nyc))

'2022-11-08 19:51:00-05:00'

In [18]:
def utc_to_local(date, utc, tz):
    
    dt = datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ")
    utc_time = utc.localize(dt)
    
    return str(utc_time.astimezone(nyc))
    

In [22]:
nyc_dates = [utc_to_local(i, utc, nyc) for i in date_time]

In [19]:
highs = weather["STATION"][0]["OBSERVATIONS"]['air_temp_high_24_hour_set_1']
highs

[None,
 None,
 None,
 None,
 57.92,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [20]:
pd.Series(highs, index=date_time)

2022-11-09T00:51:00Z      NaN
2022-11-09T01:51:00Z      NaN
2022-11-09T02:51:00Z      NaN
2022-11-09T03:51:00Z      NaN
2022-11-09T04:51:00Z    57.92
2022-11-09T05:51:00Z      NaN
2022-11-09T06:51:00Z      NaN
2022-11-09T07:51:00Z      NaN
2022-11-09T08:51:00Z      NaN
2022-11-09T09:51:00Z      NaN
2022-11-09T10:51:00Z      NaN
2022-11-09T11:51:00Z      NaN
2022-11-09T12:51:00Z      NaN
2022-11-09T13:51:00Z      NaN
2022-11-09T14:51:00Z      NaN
2022-11-09T15:51:00Z      NaN
2022-11-09T16:51:00Z      NaN
2022-11-09T17:51:00Z      NaN
2022-11-09T18:51:00Z      NaN
2022-11-09T19:51:00Z      NaN
2022-11-09T20:51:00Z      NaN
2022-11-09T21:51:00Z      NaN
2022-11-09T22:51:00Z      NaN
2022-11-09T23:51:00Z      NaN
dtype: float64

In [23]:
pd.Series(highs, index=nyc_dates)

2022-11-08 19:51:00-05:00      NaN
2022-11-08 20:51:00-05:00      NaN
2022-11-08 21:51:00-05:00      NaN
2022-11-08 22:51:00-05:00      NaN
2022-11-08 23:51:00-05:00    57.92
2022-11-09 00:51:00-05:00      NaN
2022-11-09 01:51:00-05:00      NaN
2022-11-09 02:51:00-05:00      NaN
2022-11-09 03:51:00-05:00      NaN
2022-11-09 04:51:00-05:00      NaN
2022-11-09 05:51:00-05:00      NaN
2022-11-09 06:51:00-05:00      NaN
2022-11-09 07:51:00-05:00      NaN
2022-11-09 08:51:00-05:00      NaN
2022-11-09 09:51:00-05:00      NaN
2022-11-09 10:51:00-05:00      NaN
2022-11-09 11:51:00-05:00      NaN
2022-11-09 12:51:00-05:00      NaN
2022-11-09 13:51:00-05:00      NaN
2022-11-09 14:51:00-05:00      NaN
2022-11-09 15:51:00-05:00      NaN
2022-11-09 16:51:00-05:00      NaN
2022-11-09 17:51:00-05:00      NaN
2022-11-09 18:51:00-05:00      NaN
dtype: float64

In [24]:
len(date_time)

24

### More shit


In [25]:
start_day = datetime.now().replace(day=15, month=10)
start_day = start_day.strftime("%Y%m%d") + "0001" 
start_day

'202210150001'

In [26]:
today = datetime.now().strftime("%Y%m%d%H%M")
today

'202211101619'

In [27]:
final_url = f"{base_url}&stid={stid}&start={start_day}&units=temp|F&end={today}&showemptystations=1&token={token}"
final_url

'https://api.synopticdata.com/v2/stations/timeseries?&stid=knyc&start=202210150001&units=temp|F&end=202211101619&showemptystations=1&token=933a6b2f4c4e46898f8e80c91225fd04'

In [28]:
r = requests.get(final_url)

In [29]:
r_json = r.json()

In [30]:
date_time = r_json["STATION"][0]["OBSERVATIONS"]['date_time']

In [31]:
date_time

['2022-10-15T00:51:00Z',
 '2022-10-15T01:51:00Z',
 '2022-10-15T02:51:00Z',
 '2022-10-15T03:51:00Z',
 '2022-10-15T04:51:00Z',
 '2022-10-15T05:51:00Z',
 '2022-10-15T06:51:00Z',
 '2022-10-15T07:51:00Z',
 '2022-10-15T08:51:00Z',
 '2022-10-15T09:51:00Z',
 '2022-10-15T10:51:00Z',
 '2022-10-15T11:51:00Z',
 '2022-10-15T12:51:00Z',
 '2022-10-15T13:51:00Z',
 '2022-10-15T14:51:00Z',
 '2022-10-15T15:51:00Z',
 '2022-10-15T16:51:00Z',
 '2022-10-15T17:51:00Z',
 '2022-10-15T18:51:00Z',
 '2022-10-15T19:51:00Z',
 '2022-10-15T20:51:00Z',
 '2022-10-15T21:51:00Z',
 '2022-10-15T22:51:00Z',
 '2022-10-15T23:51:00Z',
 '2022-10-16T00:51:00Z',
 '2022-10-16T01:51:00Z',
 '2022-10-16T02:51:00Z',
 '2022-10-16T03:51:00Z',
 '2022-10-16T04:51:00Z',
 '2022-10-16T05:51:00Z',
 '2022-10-16T06:51:00Z',
 '2022-10-16T07:51:00Z',
 '2022-10-16T08:51:00Z',
 '2022-10-16T09:51:00Z',
 '2022-10-16T10:51:00Z',
 '2022-10-16T11:51:00Z',
 '2022-10-16T12:51:00Z',
 '2022-10-16T13:51:00Z',
 '2022-10-16T14:51:00Z',
 '2022-10-16T15:51:00Z',


In [32]:
highs = r_json["STATION"][0]["OBSERVATIONS"]['air_temp_high_24_hour_set_1']

In [33]:
highs

[None,
 None,
 None,
 None,
 66.92,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 68.0,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 64.04,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 66.02,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 55.94,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 53.96,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 

In [34]:
k = pd.Series(highs, index=date_time).dropna()
k

2022-10-15T04:51:00Z    66.92
2022-10-16T04:51:00Z    68.00
2022-10-17T04:51:00Z    64.04
2022-10-18T04:51:00Z    66.02
2022-10-19T04:51:00Z    55.94
2022-10-20T04:51:00Z    53.96
2022-10-21T04:51:00Z    57.92
2022-10-22T04:51:00Z    62.06
2022-10-23T04:51:00Z    66.02
2022-10-24T04:51:00Z    60.98
2022-10-25T04:51:00Z    60.98
2022-10-26T04:51:00Z    69.98
2022-10-27T04:51:00Z    71.96
2022-10-28T04:51:00Z    64.94
2022-10-29T04:51:00Z    55.94
2022-10-30T04:51:00Z    57.92
2022-10-31T04:51:00Z    62.96
2022-11-01T04:51:00Z    64.94
2022-11-02T04:51:00Z    66.92
2022-11-03T04:51:00Z    69.98
2022-11-04T04:51:00Z    66.92
2022-11-05T04:51:00Z    69.98
2022-11-06T04:51:00Z    75.92
2022-11-07T04:51:00Z    75.02
2022-11-08T04:51:00Z    77.00
2022-11-09T04:51:00Z    57.92
2022-11-10T04:51:00Z    53.06
dtype: float64

In [35]:
df = k.to_frame()
df = df.shift(-1).dropna()

In [36]:
df.columns = ["temp_float"]

In [37]:
df["temp_int"] = df["temp_float"].round().astype(int)

In [38]:
df["City"] = 'NYC'
df

,temp_float,temp_int,City
2022-10-15T04:51:00Z,68.00,68,NYC
2022-10-16T04:51:00Z,64.04,64,NYC
2022-10-17T04:51:00Z,66.02,66,NYC
2022-10-18T04:51:00Z,55.94,56,NYC
2022-10-19T04:51:00Z,53.96,54,NYC
2022-10-20T04:51:00Z,57.92,58,NYC
2022-10-21T04:51:00Z,62.06,62,NYC
2022-10-22T04:51:00Z,66.02,66,NYC
2022-10-23T04:51:00Z,60.98,61,NYC
2022-10-24T04:51:00Z,60.98,61,NYC


In [203]:
conn = sqlite3.connect('../data/data.db')
cursor = conn.cursor()

In [181]:
cursor.execute("""CREATE TABLE IF NOT EXISTS observed_temp
    (ID INT PRIMARY KEY,
    CITY TEXT NOT NULL,
    DATE TEXT NOT NULL,
    TEMPERATURE INTEGER NOT NULL);""")

In [204]:
insert_query = """INSERT INTO observed_temp
               (CITY, DATE, TEMPERATURE)
               VALUES (?, ?, ?)"""

In [185]:
df["date"] = df.index

In [201]:
to_write = df[["City", "date", "temp_int"]].values.tolist()
to_write

[['NYC', '2022-10-15T04:51:00Z', 68],
 ['NYC', '2022-10-16T04:51:00Z', 64],
 ['NYC', '2022-10-17T04:51:00Z', 66],
 ['NYC', '2022-10-18T04:51:00Z', 56],
 ['NYC', '2022-10-19T04:51:00Z', 54],
 ['NYC', '2022-10-20T04:51:00Z', 58],
 ['NYC', '2022-10-21T04:51:00Z', 62],
 ['NYC', '2022-10-22T04:51:00Z', 66],
 ['NYC', '2022-10-23T04:51:00Z', 61],
 ['NYC', '2022-10-24T04:51:00Z', 61],
 ['NYC', '2022-10-25T04:51:00Z', 70],
 ['NYC', '2022-10-26T04:51:00Z', 72],
 ['NYC', '2022-10-27T04:51:00Z', 65],
 ['NYC', '2022-10-28T04:51:00Z', 56],
 ['NYC', '2022-10-29T04:51:00Z', 58],
 ['NYC', '2022-10-30T04:51:00Z', 63],
 ['NYC', '2022-10-31T04:51:00Z', 65],
 ['NYC', '2022-11-01T04:51:00Z', 67],
 ['NYC', '2022-11-02T04:51:00Z', 70],
 ['NYC', '2022-11-03T04:51:00Z', 67],
 ['NYC', '2022-11-04T04:51:00Z', 70],
 ['NYC', '2022-11-05T04:51:00Z', 76],
 ['NYC', '2022-11-06T04:51:00Z', 75]]

In [205]:
cursor.executemany(insert_query, to_write)

In [206]:
conn.commit()

In [207]:
conn.close()

## Updating

Okay the main issue is that a given day's max temp is sent out on the next day.

In [39]:
conn = sqlite3.connect('../data/data.db')
cursor = conn.cursor()

In [40]:
most_recent_day = """SELECT date(DATE) FROM observed_temp
ORDER BY date(DATE) desc
LIMIT 1;
"""

In [47]:
cursor.execute(most_recent_day)

In [48]:
start_date = cursor.fetchall()[0][0]

In [49]:
conn.close()

In [53]:
start_date_to_send = start_date.replace("-", "")+"0001"

In [55]:
final_url = f"{base_url}&stid={stid}&start={start_date_to_send}&units=temp|F&end={today}&showemptystations=1&token={token}"
final_url

'https://api.synopticdata.com/v2/stations/timeseries?&stid=knyc&start=202211060001&units=temp|F&end=202211101619&showemptystations=1&token=933a6b2f4c4e46898f8e80c91225fd04'

In [7]:
def date_dataframe(final_url):
    r = requests.get(final_url)
    if not r.ok:
        print("Error")
        return
    
    r_json = r.json()
    
    date_time = r_json["STATION"][0]["OBSERVATIONS"]['date_time']
    highs = r_json["STATION"][0]["OBSERVATIONS"]['air_temp_high_24_hour_set_1']
    k = pd.Series(highs, index=date_time).dropna()
    df = k.to_frame()
    df = df.shift(-1).dropna()
    df.columns = ["temp_float"]
    df["temp_int"] = df["temp_float"].round().astype(int)
    return df
    
df = date_dataframe(final_url)

### Trim off the most recent day

In [88]:
df

,temp_float,temp_int
2022-11-06T04:51:00Z,75.02,75
2022-11-07T04:51:00Z,77.00,77
2022-11-08T04:51:00Z,57.92,58
2022-11-09T04:51:00Z,53.06,53


In [85]:
pd.to_datetime(df.index)

DatetimeIndex(['2022-11-06 04:51:00+00:00', '2022-11-07 04:51:00+00:00',
               '2022-11-08 04:51:00+00:00', '2022-11-09 04:51:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [86]:
pd.to_datetime(start_date)

Timestamp('2022-11-06 00:00:00')

In [104]:
def prep_to_write(df, city, start_date):
    
    df["city"] = city
    df["date"] = df.index
    to_write = df[["city", "date", "temp_int"]].values.tolist()
    
    final_to_return = []
    for row in to_write:
        date = row[1].split("T")[0]
        if date > start_date:
            final_to_return.append([row[0], date, row[2]])
    
    return final_to_return


to_write = prep_to_write(df, "NYC", start_date)
to_write

[['NYC', '2022-11-07', 77],
 ['NYC', '2022-11-08', 58],
 ['NYC', '2022-11-09', 53]]

In [105]:
def write_observed_temps(to_write):
    
    conn = sqlite3.connect('../data/data.db')
    cursor = conn.cursor()
    
    insert_query = """INSERT INTO observed_temp
               (CITY, DATE, TEMPERATURE)
               VALUES (?, ?, ?)"""
    
    cursor.executemany(insert_query, to_write)
    
    conn.commit()
    conn.close()
    
write_observed_temps(to_write)

### Now let's init Chicago

`final_url = f"{base_url}&stid={stid}&start={start_date_to_send}&units=temp|F&end={today}&showemptystations=1&token={token}"
final_url`

In [108]:
start = '2022-10-15'
end = '2022-11-04'

In [119]:
city_to_stid = {
    "NYC":"knyc",
    "CHI":"kmdw"
}

synoptic_base_url = "https://api.synopticdata.com/v2/stations/timeseries?"

In [111]:
def format_date(date):
    """YYYY-MM-DD -> YYYYMMDD0001"""
    
    date_now = date.replace("-", "")
    
    return date_now + "0001"

In [113]:
def get_token():
    config = configparser.ConfigParser()
    config.read("../config")
    return config["synoptic"]["token"]

In [117]:
def format_url(city, start, end):
    """
    Parameters:
    city - string: either 'NYC' or 'CHI'
    start - a date in format YYYY-MM-DD
    end - string: either 'today' or date in format YYYY-MM-DD"""
    
    stid = city_to_stid[city]
    
    start = format_date(start)
    
    if end == 'today':
        end =  datetime.now().strftime("%Y%m%d%H%M")
    else:
        end = format_date(end)
        
    token = get_token()
    
    final_url = f"{synoptic_base_url}&stid={stid}&start={start}&units=temp|F&end={end}&showemptystations=1&token={token}"
    
    return final_url
    
final_url = format_url("CHI", start, end)
final_url

'https://api.synopticdata.com/v2/stations/timeseries?&stid=kmdw&start=202210150001&units=temp|F&end=202211040001&showemptystations=1&token=933a6b2f4c4e46898f8e80c91225fd04'

In [118]:
date_dataframe(final_url)

,temp_float,temp_int
2022-10-15T05:53:00Z,55.04,55
2022-10-16T05:53:00Z,57.02,57
2022-10-17T05:53:00Z,44.96,45
2022-10-18T05:53:00Z,50.00,50
2022-10-19T05:53:00Z,50.00,50
2022-10-20T05:53:00Z,62.96,63
2022-10-21T05:53:00Z,78.08,78
2022-10-22T05:53:00Z,82.04,82
2022-10-23T05:53:00Z,80.96,81
2022-10-24T05:53:00Z,78.98,79


In [2]:
import sys
sys.path.append("..")

import src.scraper as scraper

In [3]:
start = '2022-10-15'
end = '2022-11-04'

In [4]:
final_url = scraper.format_url("CHI", start, end)
final_url

'https://api.synopticdata.com/v2/stations/timeseries?&stid=kmdw&start=202210150001&units=temp|F&end=202211040001&showemptystations=1&token=933a6b2f4c4e46898f8e80c91225fd04'

In [5]:
df = scraper.date_dataframe(final_url)
df

,temp_float,temp_int
2022-10-15T05:53:00Z,55.04,55
2022-10-16T05:53:00Z,57.02,57
2022-10-17T05:53:00Z,44.96,45
2022-10-18T05:53:00Z,50.00,50
2022-10-19T05:53:00Z,50.00,50
2022-10-20T05:53:00Z,62.96,63
2022-10-21T05:53:00Z,78.08,78
2022-10-22T05:53:00Z,82.04,82
2022-10-23T05:53:00Z,80.96,81
2022-10-24T05:53:00Z,78.98,79


In [7]:
df = date_dataframe(final_url)
df

NameError: name 'date_dataframe' is not defined

In [3]:
import src.database as database

In [7]:
to_write = database.prep_to_write(df, "CHI", "0000-00-00")
to_write

[['CHI', '2022-10-15', 55],
 ['CHI', '2022-10-16', 57],
 ['CHI', '2022-10-17', 45],
 ['CHI', '2022-10-18', 50],
 ['CHI', '2022-10-19', 50],
 ['CHI', '2022-10-20', 63],
 ['CHI', '2022-10-21', 78],
 ['CHI', '2022-10-22', 82],
 ['CHI', '2022-10-23', 81],
 ['CHI', '2022-10-24', 79],
 ['CHI', '2022-10-25', 71],
 ['CHI', '2022-10-26', 57],
 ['CHI', '2022-10-27', 54],
 ['CHI', '2022-10-28', 61],
 ['CHI', '2022-10-29', 67],
 ['CHI', '2022-10-30', 67],
 ['CHI', '2022-10-31', 61],
 ['CHI', '2022-11-01', 70],
 ['CHI', '2022-11-02', 74]]

In [8]:
database.write_high_temps(to_write)

### And get the most recent ones

In [6]:
start_date = database.most_recent_high_date("CHI")
start_date

'2022-11-02'

In [8]:
final_url = scraper.format_url("CHI", start_date, "today")
final_url

'https://api.synopticdata.com/v2/stations/timeseries?&stid=kmdw&start=202211020001&units=temp|F&end=202211101758&showemptystations=1&token=933a6b2f4c4e46898f8e80c91225fd04'

In [9]:
df = scraper.date_dataframe(final_url)
df

,temp_float,temp_int
2022-11-02T05:53:00Z,73.94,74
2022-11-03T05:53:00Z,75.02,75
2022-11-04T05:53:00Z,75.92,76
2022-11-05T05:53:00Z,71.06,71
2022-11-06T05:53:00Z,64.04,64
2022-11-07T05:53:00Z,57.92,58
2022-11-08T05:53:00Z,53.96,54
2022-11-09T05:53:00Z,69.98,70


In [10]:
to_write = database.prep_to_write(df, "CHI", start_date)
to_write

[['CHI', '2022-11-03', 75],
 ['CHI', '2022-11-04', 76],
 ['CHI', '2022-11-05', 71],
 ['CHI', '2022-11-06', 64],
 ['CHI', '2022-11-07', 58],
 ['CHI', '2022-11-08', 54],
 ['CHI', '2022-11-09', 70]]

In [11]:
database.write_high_temps(to_write)